In [4]:

import os
import json
import ollama
from langchain_openai import ChatOpenAI
from langchain_mistralai import ChatMistralAI
from langchain.schema import HumanMessage
from Utils.Prompt import planner_no_route_agent, planner_route_OP_agent

MISTRAL_API_KEY = os.getenv('MISTRAL_API_KEY')
OPENAI_API_KEY = os.getenv('OPEN_AI_API')

class SolePlanning:
    def __init__(self,
                 planner_llm) -> None:
        
        self.planner_llm = planner_llm

        if self.planner_llm == 'gpt-4o-2024-11-20':
            self.llm = ChatOpenAI(temperature=0,
                        model_name='gpt-4o-2024-11-20',
                        openai_api_key=OPENAI_API_KEY)

        if self.planner_llm == 'mistral-large-2411':
            self.llm = ChatMistralAI(
                model="mistral-large-2411",
                temperature=0,
                mistral_api_key = MISTRAL_API_KEY
            )
    def createPlan(self, prompt):
        if self.planner_llm == 'gpt-4o-2024-11-20':
            request = self.llm.invoke([HumanMessage(prompt)]).content
            return request
        if self.planner_llm == 'mistral-large-2411':
            request = self.llm.invoke(prompt).content
            return request

        if self.planner_llm == 'llama3.1':
            request = ollama.generate(model='llama3.1', prompt=prompt, options={'num_ctx': 70000})['response']
            return request


if __name__ == '__main__':
    modelList = ['gpt4o','mistral','llama318b']
    model_map = {'gpt4o': 'gpt-4o-2024-11-20','mistral':'mistral-large-2411','llama318b':'llama3.1'}
    taskList = ['allDataNoRoute', 'allDataRouteOP', 'filteredDataRouteOP','baseData']

    #just choose the model and the task
    #choose model
    model = modelList[1]
    planner_llm = model_map[model]
    print("the model you choose is: ", model, " The planner LLM is: ", planner_llm)
    # choose task
    task = taskList[2]
    print("The task you choose is: ", task)
    #number of plan
    #numPlan = 300
    #print("You will run: " + str(numPlan) + " plans")
    

    

    #load data (all/filtered)
    if 'all' in task:
        with open(f'Dataset/gpt4o/all_data.json', 'r') as file:
            given_information = json.load(file)
    elif 'filtered' in task:
        with open(f'Dataset/gpt4o/filtered_data.jsonl', 'r') as file:
            given_informations = [json.loads(line.strip()) for line in file]
    else:
        with open ('Dataset/Base/all_base_data.json', 'r') as file:
            given_information = json.load(file)
    
    #iterate through all human querys
    plans = []
    with open ('Prompts/humanQuerys.jsonl', 'r') as file:
            humanquerys = [json.loads(line.strip()) for line in file]
    for i in range(100):
        #load human query, allData only 100 plans, filteredData 500 plans.
        query = humanquerys[i]['query']
        #print(query)
        
    
        #load prompt agent, and make the prompt
        if task == 'allDataNoRoute':
            prompt_agent = planner_no_route_agent
            prompt = prompt_agent.format(given_information = given_information, query = query)
            #print(prompt)
        if task == 'allDataRouteOP':
            prompt_agent = planner_route_OP_agent
            prompt = prompt_agent.format(given_information = given_information, query = query)
            #print(prompt)

        if task == 'filteredDataRouteOP':
            given_information = given_informations[i]['filtered_data']
            prompt_agent = planner_route_OP_agent
            prompt = prompt_agent.format(given_information = given_information, query = query)
            #print(prompt)

        if task == 'baseData':
            prompt_agent = planner_no_route_agent
            prompt = prompt_agent.format(given_information = given_information, query = query)
        

        #actual inference
        solePlanning = SolePlanning(planner_llm=planner_llm)
        plan = {"index": i+1, "plan": solePlanning.createPlan(prompt)}
        plans = [plan]

        with open (f'Output/{model}/plans/{task}.jsonl', 'a') as file:
            for plan in plans:
                file.write(json.dumps(plan) + '\n')
        if(i%20 == 0):
            print(f"we saved {i+1}th plans" )

the model you choose is:  mistral  The planner LLM is:  mistral-large-2411
The task you choose is:  filteredDataRouteOP
we saved 1th plans
we saved 21th plans
we saved 41th plans
we saved 61th plans
we saved 81th plans
